At the end of the experiment, you will be able to:

* load the data into RayDMatrix
* train the XGBoost Ray model and save it
* tune the Hyperparameters using Ray tune

## Introduction

Compute demands for machine learning (ML) training have grown 10x every
18 months since 2010. Over the same time period, the compute capabilities of
AI accelerators such as GPUs and TPUs have less than doubled. This means
that every year and a half organizations need 5x more AI accelerators/nodes
to train the latest ML models and leverage cutting edge ML capabilities.
Distributed computing is the only way to meet these requirements.

While solutions such as AWS SageMaker and GCP Vertex AI have emerged
to help organizations deal with scaling AI workloads, these solutions put
significant constraints on how applications are developed and which libraries
they can use. This makes it difficult to keep up with the latest models and
algorithms, and freely integrate with the rapidly evolving open ML ecosystem.

Ray, addresses these challenges head on by
allowing ML engineers and developers to scale their workloads effortlessly
from their laptops to the cloud without the need to build complex compute
infrastructures.

### **Ray**

Ray is an open-source unified framework for scaling AI and Python applications like machine learning. It provides the compute layer for parallel processing and reduce the need of a distributed systems expert. Ray minimizes the complexity of running your distributed individual and end-to-end machine learning workflows with these components:

- Scalable libraries for common machine learning tasks such as data preprocessing, distributed training, hyperparameter tuning, reinforcement learning, and model serving.

- Pythonic distributed computing primitives for parallelizing and scaling Python applications.

- Integrations and utilities for integrating and deploying a Ray cluster with existing tools and infrastructure such as Kubernetes, AWS, GCP, and Azure.
<br>

Some common ML workloads that individuals, organizations, and companies leverage Ray to build their AI applications include:

- Batch inference on CPUs and GPUs
- Parallel training
- Model serving
- Distributed training of large models
- Parallel hyperparameter tuning experiments
- Reinforcement learning
- ML platform


### **Ray framework**



Ray's unified compute framework consists of three layers:

- ***Ray AI Libraries:*** An open-source, Python, domain-specific set of libraries that equip ML engineers, data scientists, and researchers with a scalable and unified toolkit for ML applications.

- ***Ray Core:*** An open-source, Python, general purpose, distributed computing library that enables ML engineers and Python developers to scale Python applications and accelerate machine learning workloads.

- ***Ray Clusters:*** A set of worker nodes connected to a common Ray head node. Ray clusters can be fixed-size, or they can autoscale up and down according to the resources requested by applications running on the cluster.

<br>

Each of Ray's five native libraries distributes a specific ML task:

- **`Data`**: Scalable, framework-agnostic data loading and transformation across training, tuning, and prediction

- **`Train`**: Distributed multi-node and multi-core model training with fault tolerance that integrates with popular training libraries

- **`Tune`**: Scalable hyperparameter tuning to optimize model performance

- **`Serve`**: Scalable and programmable serving to deploy models for online inference, with optional microbatching to improve performance

- **`RLlib`**: Scalable distributed reinforcement learning workloads


Find the official Ray website [here](https://www.ray.io/), and its documentation [here](https://docs.ray.io/en/latest/ray-overview/index.html).

### Install necessary packages

In [ ]:
!pip -q install ray
!pip -q install xgboost_ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 2.4 MB/s eta 0:00:00


### Import necessary libraries

In [ ]:
from sklearn.datasets import load_breast_cancer
import xgboost as xgb
from xgboost_ray import RayDMatrix, RayParams, train, predict
from ray import tune

### Load the data

In [ ]:
train_x, train_y = load_breast_cancer(return_X_y=True)

### XGBoost-Ray uses the same API as core XGBoost

There are only two differences:

* Instead of using a `xgboost.DMatrix`, it uses `xgboost_ray.RayDMatrix` object

* There is an additional `ray_params` parameter that is used to configure distributed training

In [ ]:
train_set = RayDMatrix(train_x, train_y)

### Train the XGBoost Ray model and save it

In [ ]:
evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb_ray")

2023-11-25 15:36:51,140	INFO worker.py:1673 -- Started a local Ray instance.
2023-11-25 15:36:56,597	INFO main.py:1140 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2023-11-25 15:37:03,088	INFO main.py:1191 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=3176) [15:37:03] task [xgboost.ray]:133149315482336 got new rank 0
2023-11-25 15:37:07,730	INFO main.py:1708 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 12.86 seconds (4.62 pure XGBoost training time).
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:37:07] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


### Final training error

In [ ]:
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

Final training error: 0.0053


### Train the XGBoost model

A simple prediction example. The data will be split across two actors. The result array will integrate this data in the correct order.

In [ ]:
dpred =RayDMatrix(train_x, train_y)
bst = xgb.Booster(model_file="model.xgb_ray")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

2023-11-25 15:54:59,979	INFO main.py:1758 -- [RayXGBoost] Created 2 remote actors.
2023-11-25 15:55:06,254	INFO main.py:1775 -- [RayXGBoost] Starting XGBoost prediction.


[0.09144595 0.05673993 0.03008196 0.10851309 0.09144595 0.1226117
 0.03008196 0.03145875 0.03795947 0.09715987 0.11452682 0.03008196
 0.03008196 0.04623247 0.06943818 0.03008196 0.03738927 0.03008196
 0.03008196 0.97716796 0.98008084 0.98008084 0.0675434  0.03008196
 0.03008196 0.03620729 0.03974995 0.03008196 0.03008196 0.09527509
 0.03008196 0.03008196 0.03008196 0.03008196 0.03008196 0.03008196
 0.04384386 0.95125544 0.26743954 0.05803299 0.5169782  0.30979052
 0.03008196 0.03145875 0.11796542 0.03620729 0.98008084 0.04324226
 0.98008084 0.93069357 0.9776062  0.9790822  0.9698948  0.03261407
 0.05082424 0.9698948  0.03008196 0.03145875 0.9790822  0.9698948
 0.9731866  0.98008084 0.03008196 0.98008084 0.03145875 0.03145875
 0.9700466  0.9698948  0.86911684 0.9698948  0.03008196 0.98008084
 0.03008196 0.26402172 0.98008084 0.03008196 0.9691353  0.05734929
 0.03958243 0.98008084 0.9700466  0.8453193  0.03008196 0.03008196
 0.98008084 0.03008196 0.14372692 0.03008196 0.9427549  0.922841

## Hyperparameter Optimization with Ray

1. Put the XGBoost-Ray training call into a function accepting parameter configurations (`train_model` in the example below)
2. Create a RayParams object

3. Define the parameter search space (`config` dictionary)

4. Call `tune.run()`:
    * The metric parameter should contain the metric to optimized. Usually this consists of the prefix passed to the evals argument of `xgboost_ray.train()`, and an `eval_metric` passed in the XGBoost parameters (train-error in the example below)

    * The mode should either be min or max, depending on whether the metric is to be minimized or maximized

    * The `resources_per_actor` should be set using `ray_params.get_tune_resources()`

### Set the number of processes and the number of units per process

In [ ]:
num_actors = 4
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

### Define a function to train the XGBoost Ray model and save it

In [ ]:
def train_model(config):
    evals_result = {}
    bst = train(
    params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
        # params=config,
        # dtrain=train_set,
        # evals_result=evals_result,
        # evals=[(train_set, "train")],
        # verbose_eval=False,
        # ray_params=ray_params)
    bst.save_model("model.xgb_ray")

### Specify the hyperparameter search space.

In [ ]:
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

### Train and display the results

Ray Tune uses Ray to start multiple distributed trials with different hyperparameter configurations. If used with XGBoost-Ray, these trials will then start their own distributed training jobs.

XGBoost-Ray automatically reports evaluation results back to Ray Tune.

documentation [here](https://xgboost.readthedocs.io/en/stable/tutorials/ray.html#hyperparameter-optimization)


Make sure to use the `get_tune_resources` method to set the `resources_per_trial`

This will make sure that each trial has the necessary resources available to start their distributed training jobs.

In [ ]:
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2023-11-25 15:57:43,635	INFO tune.py:595 -- [output] This will use the new output engine with verbosity 2. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-11-25 15:57:43,653	INFO tensorboardx.py:178 -- pip install "ray[tune]" to see TensorBoard files.
2023-11-25 15:57:43,656	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


+--------------------------------------------------------------------+
| Configuration for experiment     train_model_2023-11-25_15-57-43   |
+--------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator             |
| Scheduler                        FIFOScheduler                     |
| Number of trials                 4                                 |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_2023-11-25_15-57-43

Trial status: 4 PENDING
Current time: 2023-11-25 15:57:43. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+------------------------------------------------------------------------------+
| Trial name                status             eta     subsample     max_depth |
+------------------------------------------------------------------------------+
| train_model_5e852_00000   PENDING    0.0979471       

2023-11-25 15:58:44,040	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 4.0 CPUs and 0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


Trial status: 4 PENDING
Current time: 2023-11-25 15:58:43. Total running time: 1min 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+------------------------------------------------------------------------------+
| Trial name                status             eta     subsample     max_depth |
+------------------------------------------------------------------------------+
| train_model_5e852_00000   PENDING    0.0979471        0.772874             1 |
| train_model_5e852_00001   PENDING    0.000179901      0.983443             1 |
| train_model_5e852_00002   PENDING    0.000161912      0.855457             2 |
| train_model_5e852_00003   PENDING    0.00651198       0.508894             7 |
+------------------------------------------------------------------------------+
Trial status: 4 PENDING
Current time: 2023-11-25 15:59:13. Total running time: 1min 30s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+------------------------------------------------------------------------------+
| Trial na

2023-11-25 15:59:44,055	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 4.0 CPUs and 0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


...

Trial status: 4 PENDING
Current time: 2023-11-25 16:40:45. Total running time: 43min 2s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+------------------------------------------------------------------------------+
| Trial name                status             eta     subsample     max_depth |
+------------------------------------------------------------------------------+
| train_model_5e852_00000   PENDING    0.0979471        0.772874             1 |
| train_model_5e852_00001   PENDING    0.000179901      0.983443             1 |
| train_model_5e852_00002   PENDING    0.000161912      0.855457             2 |
| train_model_5e852_00003   PENDING    0.00651198       0.508894             7 |
+------------------------------------------------------------------------------+
Trial status: 4 PENDING
Current time: 2023-11-25 16:41:15. Total running time: 43min 32s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+------------------------------------------------------------------------------+
| Trial 

2023-11-25 16:41:45,200	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 4.0 CPUs and 0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


Trial status: 4 PENDING
Current time: 2023-11-25 16:41:45. Total running time: 44min 2s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+------------------------------------------------------------------------------+
| Trial name                status             eta     subsample     max_depth |
+------------------------------------------------------------------------------+
| train_model_5e852_00000   PENDING    0.0979471        0.772874             1 |
| train_model_5e852_00001   PENDING    0.000179901      0.983443             1 |
| train_model_5e852_00002   PENDING    0.000161912      0.855457             2 |
| train_model_5e852_00003   PENDING    0.00651198       0.508894             7 |
+------------------------------------------------------------------------------+


2023-11-25 16:41:54,400	WARNING tune.py:186 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-11-25 16:41:54,419	WARNING tune.py:1062 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2023-11-25 16:41:54,432	WARNING experiment_analysis.py:185 -- Failed to fetch metrics for 4 trial(s):
- train_model_5e852_00000: FileNotFoundError('Could not fetch metrics for train_model_5e852_00000: both result.json and progress.csv were not found at /root/ray_results/train_model_2023-11-25_15-57-43/train_model_5e852_00000_0_eta=0.0979,max_depth=1,subsample=0.7729_2023-11-25_15-57-43')
- train_model_5e852_00001: FileNotFoundError('Could not fetch metrics for train_model_5e852_00001: both result.json and progress.csv were not found at /root/ray_results/train_model_2023-11-25_15-57-43/trai

Trial status: 4 PENDING
Current time: 2023-11-25 16:41:54. Total running time: 44min 10s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+------------------------------------------------------------------------------+
| Trial name                status             eta     subsample     max_depth |
+------------------------------------------------------------------------------+
| train_model_5e852_00000   PENDING    0.0979471        0.772874             1 |
| train_model_5e852_00001   PENDING    0.000179901      0.983443             1 |
| train_model_5e852_00002   PENDING    0.000161912      0.855457             2 |
| train_model_5e852_00003   PENDING    0.00651198       0.508894             7 |
+------------------------------------------------------------------------------+

Best hyperparameters None
